In [1]:
import pandas as pd
import scipy.stats
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
#Load data
df = pd.read_csv('D:/My stuff/School/Master/Master Thesis/Data/Kaggle/Home credit/application_train.csv', na_values = ["", "NA", "XNA"])
var_info = pd.read_excel('D:/My stuff/School/Master/Master Thesis/Data/Kaggle/Home credit/HomeCredit_columns_description.xlsx')

In [3]:
#Check shape
df.shape

(307511, 122)

# Data adjustments

In [4]:
#Drop the ID column
df.drop('SK_ID_CURR', axis = 1, inplace = True)
var_info.drop(0, axis = 0, inplace = True)

In [5]:
#Rename columns for clarity
df.rename(columns = {'TARGET' : 'Target'}, inplace = True)

# Drop variables with low fill rate

In [6]:
#Drop variables with too many missing values
miss_perc = df.isna().sum() / df.shape[0] > 0.2
print(f'Dropped {sum(miss_perc)} columns with percentage of missings higher than 0.2')
df = df.loc[:, np.invert(miss_perc)]
var_info = var_info.loc[np.invert(miss_perc.values), :]

Dropped 50 columns with percentage of missings higher than 0.2


# Retain only categorical features with two unique values

In [7]:
#Get number of unique values
n_unique = df.nunique() #Get number of unique values (missings disregarded)
cat_to_drop = (n_unique != 2) & (var_info['Type'] == 'categorical').values
print(f'Dropping {sum(cat_to_drop)} categorical features with more or less than 2 unique values')
df = df.loc[:, np.invert(cat_to_drop)] #Drop the features
var_info = var_info.loc[np.invert(cat_to_drop.values), :]

Dropping 10 categorical features with more or less than 2 unique values


# Create dummy variables

In [8]:
#Create dummy variables
df.rename(columns = {'NAME_CONTRACT_TYPE' : 'CASH_LOANS', 'CODE_GENDER' : 'FEMALE'}, inplace = True) #Rename two columns for clarity
var_info['Row'].replace({'NAME_CONTRACT_TYPE' : 'CASH_LOANS', 'CODE_GENDER' : 'FEMALE'}, inplace = True) #Rename in metadata as well
df['CASH_LOANS'] = df['CASH_LOANS'].map({'Cash loans' : 1, 'Revolving loans' : 0}, na_action = 'ignore')
df['FEMALE'] = df['FEMALE'].map({'F' : 1, 'M' : 0}, na_action = 'ignore')
df['FLAG_OWN_CAR'] = df['FLAG_OWN_CAR'].map({'Y' : 1, 'N' : 0}, na_action = 'ignore')
df['FLAG_OWN_REALTY'] = df['FLAG_OWN_REALTY'].map({'Y' : 1, 'N' : 0}, na_action = 'ignore')

# Drop correlated

In [9]:
#Specifications
dep_var = 'Target'
thres = 0.75 #Arbitrary for now

In [10]:
#Calculate the correlations
indep_vars = list(set(df.columns.to_list()) - set([dep_var])) #Specify the list of independent variables
corr_mat = df.corr() #Calculate the correlation matrix
dep_cor = corr_mat[dep_var].copy() #Extract the correlations with the dependent variable
corr_mat.drop(dep_var, axis = 0) #Drop the row with correlations with the dependent variable
corr_mat.drop(dep_var, axis = 1) #Drop the column with the correlations with the dependent variable
corr_mat.values[np.tril_indices_from(corr_mat.values)] = np.nan #Leave only the upper triangle
corr_mat = corr_mat.unstack().dropna().reset_index() #Unstack to a table
corr_mat.columns = ['Var 1', 'Var 2', 'Corr'] #Rename columns for clarity
corr_mat = corr_mat.loc[np.argsort(-corr_mat['Corr'].abs(), ), :].reset_index(drop = True) #Sort in absolute values

In [11]:
#Drop correlated features
n_dropped = 0 #Initiate the number of dropped features
for i in corr_mat.index[corr_mat['Corr'].abs() >= thres]: #Loop through all correlations higher than threshold (in absolute value)
    var1, var2 = corr_mat.loc[i, 'Var 1'], corr_mat.loc[i, 'Var 2'] #Store variable names
    if (var1 not in indep_vars) | (var2 not in indep_vars):
        continue #Skip the iteration if one of the variables has already been disregarded
    var_types = var_info.loc[var_info['Row'] == var1, 'Type'].values[0], var_info.loc[var_info['Row'] == var2, 'Type'].values[0] #Store variable types
    if var_types.count('categorical') == 1: #If only one of the variables is categorical, retain the numerical one
        var_to_drop = [var1, var2][var_types.index('categorical')] #Store the categorical variable to drop
        not_dropped_var = [var1, var2][var_types.index('numerical')] #Store the numerical variable to retain
    else:
        min_id = np.argmin(dep_cor[corr_mat.loc[i, ['Var 1', 'Var 2']].values].abs()) #Find the id of the variable with the smallest absolute correlation with the dependent variable
        var_to_drop = corr_mat.loc[i, f'Var {min_id + 1}'] #Get the name of the variable to be dropped
        not_dropped_var = corr_mat.loc[i, f'Var {abs(min_id - 2)}'] #Get the name of the variable that was not dropped (for logging purposes)
    indep_vars.remove(var_to_drop) #Drop the variable
    print(f'Variable {var_to_drop} dropped due to correlation of {corr_mat.loc[i, "Corr"]:.2%} with {not_dropped_var}')
    n_dropped += 1
print(f'Dropped {n_dropped} features due to high correlation')

Variable FLAG_EMP_PHONE dropped due to correlation of -99.98% with DAYS_EMPLOYED
Variable OBS_60_CNT_SOCIAL_CIRCLE dropped due to correlation of 99.85% with OBS_30_CNT_SOCIAL_CIRCLE
Variable AMT_CREDIT dropped due to correlation of 98.70% with AMT_GOODS_PRICE
Variable CNT_FAM_MEMBERS dropped due to correlation of 87.92% with CNT_CHILDREN
Variable LIVE_REGION_NOT_WORK_REGION dropped due to correlation of 86.06% with REG_REGION_NOT_WORK_REGION
Variable DEF_60_CNT_SOCIAL_CIRCLE dropped due to correlation of 86.05% with DEF_30_CNT_SOCIAL_CIRCLE
Variable LIVE_CITY_NOT_WORK_CITY dropped due to correlation of 82.56% with REG_CITY_NOT_WORK_CITY
Variable AMT_ANNUITY dropped due to correlation of 77.51% with AMT_GOODS_PRICE
Dropped 8 features due to high correlation


In [12]:
#Filter out dropped variables
df = df.loc[:, indep_vars + [dep_var]]
var_info = var_info.loc[var_info['Row'].isin(indep_vars + [dep_var]).values, :]

In [13]:
#Calculate VIF
#vif_dict = {val:round(variance_inflation_factor(df.loc[:, indep_vars].dropna(), idx), 2) for idx, val in enumerate(indep_vars)}

In [14]:
#Check VIF values
# for i in vif_dict:
#     if (vif_dict[i] > 10) | pd.isna(vif_dict[i]):
#         print(i, vif_dict[i])

In [15]:
#Drop FLAG_DOCUMENT_2 and recalculate VIF
indep_vars.remove('FLAG_DOCUMENT_2')
# vif_dict = {val:round(variance_inflation_factor(df.loc[:, indep_vars].dropna(), idx), 2) for idx, val in enumerate(indep_vars)}
# for i in vif_dict:
#     if (vif_dict[i] > 10) | pd.isna(vif_dict[i]):
#         print(i, vif_dict[i])

In [16]:
#Drop FLAG_MOBIL and recalculate VIF
indep_vars.remove('FLAG_MOBIL')
# vif_dict = {val:round(variance_inflation_factor(df.loc[:, indep_vars].dropna(), idx), 2) for idx, val in enumerate(indep_vars)}
# for i in vif_dict:
#     if (vif_dict[i] > 10) | pd.isna(vif_dict[i]):
#         print(i, vif_dict[i])

In [17]:
#Drop FLAG_DOCUMENT_3 and recalculate VIF for all
indep_vars.remove('FLAG_DOCUMENT_3')
# vif_dict = {val:round(variance_inflation_factor(df.loc[:, indep_vars].dropna(), idx), 2) for idx, val in enumerate(indep_vars)}
# for i in vif_dict:
#     if vif_dict[i] > 10:
#         print(i, vif_dict[i])

In [18]:
#Drop FLAG_CONT_MOBILE and recalculate VIF for all
indep_vars.remove('FLAG_CONT_MOBILE')
# vif_dict = {val:round(variance_inflation_factor(df.loc[:, indep_vars].dropna(), idx), 2) for idx, val in enumerate(indep_vars)}
# for i in vif_dict:
#     if vif_dict[i] > 10:
#         print(i, vif_dict[i])

In [19]:
#Drop DAYS_BIRTH and recalculate VIF for all
indep_vars.remove('DAYS_BIRTH')
# vif_dict = {val:round(variance_inflation_factor(df.loc[:, indep_vars].dropna(), idx), 2) for idx, val in enumerate(indep_vars)}
# for i in vif_dict:
#     if vif_dict[i] > 10:
#         print(i, vif_dict[i])

In [20]:
#Drop the variables disregarded by VIF
df = df.loc[:, indep_vars + [dep_var]]
var_info = var_info.loc[var_info['Row'].isin(indep_vars + [dep_var]).values, :]

In [21]:
#Check shape after filtering
df.shape

(307511, 48)

# Remove NAs

In [22]:
#Drop NAs
df_no_nas = df.dropna()
print(f'{df_no_nas.shape[0]} out of {df.shape[0]} observations left after removing NAs ({df.shape[0] - df_no_nas.shape[0]} observations dropped)')

245155 out of 307511 observations left after removing NAs (62356 observations dropped)


# Remove categorical variables

In [23]:
#Remove categorical variables
df_no_cats = df_no_nas.loc[:, df_no_nas.nunique() > 2].copy()
df_no_cats['Target'] = df_no_nas['Target'] #Retain target

In [24]:
#Check shape
df_no_cats.shape

(245155, 19)

# Get info about final state of the data set

In [25]:
#Check shape
df_no_nas.shape

(245155, 48)

In [26]:
#Get average of the target variable
df_no_nas['Target'].mean()

0.07779160123187372

In [27]:
#Check number of categorical variables
cat_vars_final_no = sum(df_no_nas.nunique() <= 2) - 1
cat_vars_final_no

29

In [28]:
#Check number of numerical variables
df_no_nas.shape[1] - cat_vars_final_no - 1

18

# Trim outliers

In [29]:
#Trim outliers
df_no_outs = df_no_nas.copy()
for i in indep_vars:
    mask = (df_no_outs[i] >= np.percentile(df_no_outs[i], 1)) & (df_no_outs[i] <= np.percentile(df_no_outs[i], 99))
    print(f'{i}: {sum(np.invert(mask))} observations dropped')
    df_no_outs = df_no_outs.loc[mask, :]
print(f'Total observations dropped: {df_no_nas.shape[0] - df_no_outs.shape[0]}')

FLAG_WORK_PHONE: 0 observations dropped
FLAG_DOCUMENT_14: 778 observations dropped
AMT_INCOME_TOTAL: 4862 observations dropped
FLAG_DOCUMENT_19: 153 observations dropped
FLAG_DOCUMENT_21: 75 observations dropped
FLAG_OWN_REALTY: 0 observations dropped
REG_REGION_NOT_LIVE_REGION: 0 observations dropped
AMT_REQ_CREDIT_BUREAU_WEEK: 282 observations dropped
DAYS_REGISTRATION: 4749 observations dropped
EXT_SOURCE_3: 4624 observations dropped
DAYS_ID_PUBLISH: 4566 observations dropped
AMT_REQ_CREDIT_BUREAU_HOUR: 1365 observations dropped
FLAG_DOCUMENT_7: 24 observations dropped
EXT_SOURCE_2: 4474 observations dropped
DEF_30_CNT_SOCIAL_CIRCLE: 1070 observations dropped
OBS_30_CNT_SOCIAL_CIRCLE: 1873 observations dropped
FLAG_DOCUMENT_9: 743 observations dropped
CNT_CHILDREN: 379 observations dropped
FLAG_DOCUMENT_11: 739 observations dropped
FLAG_DOCUMENT_17: 64 observations dropped
FLAG_DOCUMENT_12: 0 observations dropped
FLAG_DOCUMENT_18: 1835 observations dropped
FLAG_PHONE: 0 observations

# Export

In [30]:
#Export both versions of the data set
df.to_csv('D:/My stuff/School/Master/Master Thesis/Data/Final data/HomeCredit_nas.csv', index = False)
df_no_nas.to_csv('D:/My stuff/School/Master/Master Thesis/Data/Final data/HomeCredit_main.csv', index = False)
df_no_cats.to_csv('D:/My stuff/School/Master/Master Thesis/Data/Final data/HomeCredit_no_cats.csv', index = False)
df_no_outs.to_csv('D:/My stuff/School/Master/Master Thesis/Data/Final data/HomeCredit_no_outs.csv', index = False)